In [1]:
import pickle
import os
from dotenv import load_dotenv
from openai import AsyncOpenAI
from supabase import create_client, Client
import openai
import pandas as pd

# Load the podcast data from the pickle file
with open('podcast_data.pkl', 'rb') as file:
    podcast_data = pickle.load(file)

# Load environment variables from the .env file
load_dotenv()

# Initialize OpenAI client
openai_client = AsyncOpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Initialize Supabase client
supabase: Client = create_client(
    os.getenv("SUPABASE_URL"),
    os.getenv("SUPABASE_SERVICE_KEY")
)


In [2]:
episode_number_to_match = ['042', '043', '132', '133', '200', '201']

# Create a subset of the podcast data where the episode number matches the specified values
podcast_data = podcast_data[podcast_data['episode number'].isin(episode_number_to_match)]


In [3]:



# Load environment variables from .env file
openai.api_key = os.getenv("OPENAI_API_KEY")

def generate_summary(transcript: str, existing_summary: str) -> str:
    """
    Uses GPT-4o-mini to summarize the key takeaway points from both the transcript
    and the existing summary. Returns a concise bullet-point list.
    """
    prompt_content = (
        "Below is the transcript and an existing summary for a podcast episode. "
        "Summarize the key takeaway points in bullet-point form, with each bullet representing a distinct key point. You can use multiple sentences if you need to. "
        "Make sure to include all the important details and main ideas. "
        "Do not include any additional commentary.\n\n"
        f"Transcript:\n{transcript}\n\n"
        f"Existing Summary:\n{existing_summary}"
    )
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": (
                    "You are a helpful assistant that creates clear, concise summaries of podcasts. "
                    "You think from the perspective of a listener who wants to quickly grasp the main points. "
                    "Your audience is data scientists and machine learning engineers who are listening to try to expand their skillsets and learn from the experiences of the podcasters."
                )
            },
            {
                "role": "user",
                "content": prompt_content
            }
        ],
        temperature=0.2
    )
    return response.choices[0].message.content.strip()

def add_gpt_summaries(df: pd.DataFrame) -> pd.DataFrame:
    """
    For each row in the DataFrame, sends both the transcript and the existing summary to the model for summarization,
    then stores the result in a new 'GPT Summary' column.
    """
    summaries = []
    for _, row in df.iterrows():
        transcript = row['Transcript']
        # Use the 'Summary' column if it exists; otherwise, default to an empty string.
        existing_summary = row.get('Summary', '')
        summary_text = generate_summary(transcript, existing_summary)
        summaries.append(summary_text)
    df['GPT Summary'] = summaries
    return df

# Example usage (assuming podcast_data is your DataFrame with 'Transcript' and 'Summary' columns):
podcast_data = add_gpt_summaries(podcast_data)



KeyboardInterrupt: 

In [ ]:

def generate_memorable_quotes(transcript: str, existing_summary: str) -> str:
    """
    Uses GPT-4o-mini to extract the most memorable quotes from the podcast transcript,
    explaining each quote with context and including the last time stamp for that quote.
    Returns a bullet-point list of quotes with their context.
    """
    prompt_content = (
        "Below is the transcript and an existing summary for a podcast episode. "
        "Extract the most memorable quotes from the transcript and, for each quote, "
        "provide a brief explanation of its context along with the last time stamp where that quote appears. "
        "Present the results as a bullet-point list, where each bullet contains the quote, context, and time stamp. "
        "Do not include any additional commentary.\n\n"
        f"Transcript:\n{transcript}\n\n"
        f"Existing Summary:\n{existing_summary}"
    )
    
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": (
                    "You are a helpful assistant skilled in extracting memorable quotes from podcasts. "
                    "Your output should be clear, concise, and formatted as a bullet-point list. "
                    "Your audience is data scientists and machine learning engineers who are listening to try to expand their skillsets and learn from the experiences of the podcasters."
                    "When extracting quotes, focus on the ones that are impactful, insightful, or thought-provoking. "
                    "When stating people's names, use their full names. "
                    "Each bullet should contain the quote, the speakers full name, a brief explanation of its context, and the last time stamp in the transcript where that quote occurs."
                    "Format the output as a bullet-point list like this: -'quote'\n*Speaker's full name\n*explanation\n*last time stamp"
                )
            },
            {
                "role": "user",
                "content": prompt_content
            }
        ],
        temperature=0.2
    )
    return response.choices[0].message.content.strip()

def add_gpt_memorable_quotes(df: pd.DataFrame) -> pd.DataFrame:
    """
    For each row in the DataFrame, sends both the transcript and the existing summary to the model for extracting memorable quotes,
    then stores the result in a new 'GPT Memorable Quotes' column.
    """
    quotes_list = []
    for _, row in df.iterrows():
        transcript = row['Transcript']
        # Use the 'Summary' column if it exists; otherwise, default to an empty string.
        existing_summary = row.get('GPT Summary', '')
        quotes_text = generate_memorable_quotes(transcript, existing_summary)
        quotes_list.append(quotes_text)
    df['GPT Memorable Quotes'] = quotes_list
    return df

# Example usage (assuming podcast_data is your DataFrame with 'Transcript' and 'Summary' columns):
podcast_data = add_gpt_memorable_quotes(podcast_data)


In [ ]:
podcast_quotes = podcast_data[['episode number','Title','URL','GPT Memorable Quotes','Date Published']]
